# 📝 Lab #3 (Part 7): 통신사 고객 이탈 예측 모델 해석

지금까지 배운 세 가지 모델 해석 기법(**Permutation Importance, LIME, SHAP**)을 모두 사용하여, 실제 고객 이탈(Churn) 예측 모델의 의사결정 과정을 분석하는 실습을 진행합니다.

**Telco Customer Churn 데이터셋**을 사용하여 고객 이탈 여부를 예측하는 `LightGBM` 모델을 만들고, 전역적/지역적 관점에서 모델을 심층 분석합니다.

### 과제 목표
1. `LightGBM` 분류 모델을 훈련시키고 성능을 평가합니다.
2. **Permutation Importance**를 사용해 모델의 전역적인 특성 중요도를 파악합니다.
3. **SHAP**을 사용하여 전역적(Summary Plot) 및 지역적(Force Plot) 해석을 수행합니다.
4. **LIME**을 사용하여 특정 고객(이탈 고객 vs. 비이탈 고객)의 예측 근거를 설명합니다.
5. 세 가지 기법의 결과를 종합하여 모델에 대한 분석 보고서를 작성합니다.

### 1. 데이터 준비 및 모델 훈련

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# 데이터 로드
path = '../../datasets/ml/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(path)

# --- 데이터 전처리 ---
# TotalCharges가 공백인 행 제거 및 숫자형으로 변환
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(inplace=True)

# customerID 제거
df.drop('customerID', axis=1, inplace=True)

# 이진 특성 변환
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# 범주형 특성 인코딩 (Label Encoding)
categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

# 특성과 타겟 분리
X = df.drop('Churn', axis=1)
y = df['Churn']
feature_names = X.columns.tolist()

# 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 모델 훈련
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_train, y_train)

# 모델 성능 평가
preds = lgbm.predict(X_test)
proba = lgbm.predict_proba(X_test)[:, 1]
print(f"Accuracy: {accuracy_score(y_test, preds):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, proba):.4f}")

### 2. 전역적 해석 (Global Interpretation)

#### [문제 1] Permutation Importance 계산

`lgbm` 모델과 테스트 데이터(`X_test`, `y_test`)를 사용하여 Permutation Importance를 계산하고, 상위 10개 특성을 Bar chart로 시각화하세요. (`scoring`은 `roc_auc`를 사용하세요.)

In [ ]:
from sklearn.inspection import permutation_importance

# 여기에 코드를 작성하세요

#### [문제 2] SHAP Summary Plot

`shap` 라이브러리를 사용하여 `lgbm` 모델의 `summary_plot`을 그려보세요. Permutation Importance 결과와 어떻게 다른지 비교해보세요.

In [ ]:
import shap
shap.initjs()

# 여기에 코드를 작성하세요

### 3. 지역적 해석 (Local Interpretation)

#### [문제 3] 특정 고객 예측 해석 (SHAP Force Plot)

테스트 데이터셋에서 **실제로 이탈했고(Churn=1), 모델도 이탈할 것이라고 올바르게 예측한 고객**을 한 명 골라 SHAP `force_plot`으로 예측 이유를 설명하세요.

In [ ]:
# 여기에 코드를 작성하세요

#### [문제 4] 특정 고객 예측 해석 (LIME)

테스트 데이터셋에서 **실제로 이탈하지 않았고(Churn=0), 모델도 이탈하지 않을 것이라고 올바르게 예측한 고객**을 한 명 골라 LIME으로 예측 이유를 설명하세요. (`num_features`는 6으로 설정)

In [ ]:
import lime
import lime.lime_tabular

# 여기에 코드를 작성하세요

### 4. 종합 분석

#### [문제 5] 결과 분석 및 토의

위에서 수행한 4가지 분석(Permutation Importance, SHAP Summary, SHAP Force, LIME) 결과를 바탕으로 다음 질문에 답하세요. (Markdown 셀에 답변을 작성)

1. Permutation Importance와 SHAP Summary Plot에서 공통적으로 중요하게 나타난 최상위 특성 3가지는 무엇인가요? 두 기법의 결과에 차이가 있다면 그 이유는 무엇일까요?
2. [문제 3]에서 분석한 '이탈 고객'의 이탈 예측에 가장 큰 영향을 미친 특성은 무엇이었나요? 해당 특성의 값이 어떠했기 때문에 모델이 이탈을 예측했는지 설명하세요.
3. [문제 4]에서 분석한 '비이탈 고객'의 비이탈 예측에 가장 큰 영향을 미친 특성은 무엇이었나요? 
4. 이 모델을 비즈니스에 활용한다면, 어떤 고객에게 어떤 마케팅 활동을 제안할 수 있을까요? 위 분석 결과를 근거로 아이디어를 제시해보세요.